In [3]:
import pandas as pd
import numpy as np

In [5]:
#import the files
df = pd.read_csv(r"C:\Users\harsi\Data Science\Fasal\Assignment\SampleData.csv",parse_dates=["date"],infer_datetime_format="%d/%m/%Y")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3012 entries, 0 to 3011
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   deviceId   3012 non-null   object        
 1   date       3012 non-null   datetime64[ns]
 2   month      3012 non-null   int64         
 3   hour       3012 non-null   int64         
 4   TC         3012 non-null   float64       
 5   HUM        3012 non-null   float64       
 6   LW         3012 non-null   float64       
 7   PLV2       3012 non-null   float64       
 8   farmId     3012 non-null   object        
 9   crop name  3012 non-null   object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(3)
memory usage: 235.4+ KB


In [6]:
#data quality 
check_df= df.groupby(["farmId","date"]).agg({"date":"size"})
print(check_df[check_df["date"]<24])
#check if data is populated for all dates
print(df['date'].max() - df['date'].min())
print(len(pd.unique(df['date'])))

                              date
farmId            date            
2CkGqcWtBndq5pHCh 2021-01-25    10
                  2021-05-19    21
                  2021-05-31     5
126 days 00:00:00
127


In [20]:
#There are 3 dates where the data is not captured for 24 hours but that shouldnt affect to the model
# we could do missing value imputation otherwise such as
# if TC missing replace it by average for the day
# if LW missing replace it by 0
# if PLV2 missing replace it by 0
#since we are looking at daily aggregated values, this is not a hinderance

In [21]:
#Aggregating the data at daily level
daily_data = df.groupby(["farmId","date"], as_index = False).agg(mean_TC=('TC','mean'),sum_LW=('LW','sum'),sum_PLV2=('PLV2','sum'))
daily_data = daily_data.fillna(0)

In [22]:
#Calculating the risk1 basis the daily TC & LW details
def risk_flag1(df):

    if (df['mean_TC'] <5) or (df['mean_TC'] >=30):
        return 'L'
    elif (df['mean_TC'] < 18) and (df['sum_LW'] < 14):
        return 'L'
    elif (df['mean_TC'] < 18) and (df['sum_LW'] >= 14):
        return 'M'
    elif (df['mean_TC'] < 25) and (df['sum_LW'] < 6):
        return 'L'
    elif (df['mean_TC'] < 25) and (df['sum_LW'] < 12):
        return 'M'
    elif (df['mean_TC'] < 25) and (df['sum_LW'] >= 12):
        return 'H'
    elif (df['mean_TC'] < 30) and (df['sum_LW'] < 10):
        return 'L'
    elif (df['mean_TC'] < 30) and (df['sum_LW'] >= 10):
        return 'M'
    
daily_data['Risk1'] = daily_data.apply(risk_flag1, axis = 1)
daily_data.shape

(127, 6)

In [23]:
# Rolling the data for 10 days average
rain_data= pd.DataFrame(daily_data['sum_PLV2'].rolling(10).mean())
rain_data = rain_data.fillna(0).rename(columns={'sum_PLV2': 'moving_avg_PLV2'})

In [24]:
#Calculating the risk2 basis the rainfall details

def risk_flag2(df):

    if (0.28 <= df['moving_avg_PLV2'] <3) or (df['moving_avg_PLV2'] >=25):
        return 'L'
    elif (3 <= df['moving_avg_PLV2'] <7):
        return 'M'
    elif (7 <= df['moving_avg_PLV2'] <25):
        return 'H'
    else:
        return 'U'
    
rain_data['Risk2'] = rain_data.apply(risk_flag2, axis = 1)
rain_data.shape

(127, 2)

In [25]:
#merge the two files
risk_data = pd.concat([daily_data, rain_data],axis = 1)
risk_data.shape

(127, 8)

In [26]:
# Data with Risk Flag
conditions = [(risk_data["Risk1"] == 'H')|(risk_data["Risk2"]=='H'),\
              (risk_data["Risk1"] == 'M')|(risk_data["Risk2"]=='M')]
choices1 = ['H','M']
risk_data["Risk"] = np.select(conditions, choices1, default = 'L')

#coding the risk conditions with wts for spray cals, H-2, M-1 & L-0
choices2 = [2,1]
risk_data["Risk_wt"] = np.select(conditions, choices2, default = 0)
risk_data["Risk_count"]= pd.DataFrame(risk_data['Risk_wt'].rolling(10).sum())
risk_data = risk_data.fillna(0)

In [27]:
#spray schedule
j=1
for label, row in risk_data.iterrows():
    if row['Risk_count']>=8:#if risk crosses 4H
        #Do it first time or after every 10 days
        if ((label == j-1) or (j > 9)):
            risk_data.loc[label,'spray'] ="Y"
            j = 1
        else:
            risk_data.loc[label,'spray'] ="N"
            j = j+1
    else:
        if j==15:#if risk doesn't cross 4H in last 10 days then 15th day
            risk_data.loc[label,'spray'] ="Y"
            j = 1
        else:
            risk_data.loc[label,'spray'] ="N"
            j = j+1

In [29]:
risk_data.to_csv("tmep.csv")

In [28]:
spray_schedule = risk_data[["farmId","date","Risk","spray"]]